In [39]:
## plot histogram across time points, compute entropy, 
## and plot histogram of entropy across subjects.
## xin Apr 11 2022.

import glob
import pandas as pd
import numpy as np
import re
import os
import matplotlib.pyplot as plt

## hcp data:
# input_harmonics_wavelets = 'hcp_out02_harmonics_100'
# input_time_signal = '/home/xin/Downloads/FullData_Oct26/Scan1/'
# output_dir = 'hcp_out03_spectrum_power'

## adni data:
input_dir = 'ocd_out03_spectrum_power_hc'
input_dir2 = 'ocd_out03_spectrum_power_ocd/'

output_dir = 'ocd_out04_power_histogram_and_entropy'
outfile_name_header = '/out04_entropy'


if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    
spectrum_files = glob.glob(input_dir + '/spectrum_subj*_roi_*.csv')
spectrum_files.sort()

spectrum_files2 = glob.glob(input_dir2 + '/spectrum_subj*_roi_*.csv')
spectrum_files2.sort()


In [40]:
len(spectrum_files)

5220

In [41]:
num_hc = len(spectrum_files)//116
num_ocd = len(spectrum_files2)//116

subject_info = pd.DataFrame({'group': ['hc'] * num_hc + ['ocd'] * num_ocd})
subject_info

spectrum_files += spectrum_files2

In [42]:
from math import log, e

def entropy2(labels, base=None):
    """ Computes entropy of label distribution. """

    n_labels = len(labels)

    if n_labels <= 1:
        return 0

    value,counts = np.unique(labels, return_counts=True)
    probs = counts / n_labels
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0

    ent = 0.

    # Compute entropy
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)

    return ent

nan_col = []


num_roi = 116
num_subjects = len(spectrum_files)//num_roi
entropy_list = [[None] * num_roi for _ in range(num_subjects)]

subj_id = -1
for file in spectrum_files:
    ## each file is a matrix (subjects by brain region)
    ## the values are inner product of harmonics and time course signal.
    
    print(file)
    #subj_id = re.search('(.*)spectrum_subj(.*)_roi_(.*).csv', file).group(2)
    roi = re.search('(.*)spectrum_subj(.*)_roi_(.*).csv', file).group(3)
    if int(roi)==0:
        subj_id += 1
    
    # get AD diagnosis:
    diagnosis = subject_info.iloc[int(subj_id),:]['group']
    
    spectrum = pd.read_csv(file, header = None, sep = ',')
    num_roi = spectrum.shape[1]
    
    result = []
    for col in spectrum:
        
        if spectrum[col].isna().any():
            
            print('nan column found for: ', col)
            #spectrum[col][spectrum[col].isna()] = 0
            nan_col.append(col)
            result.append(np.nan)
            #entropy = np.nan
            continue
 
        # compute entropy:
        bins = np.linspace(min(spectrum[col]), max(spectrum[col]), num = 50)
        discretized_signal = np.digitize(spectrum[col], bins)
        entropy = entropy2(discretized_signal)
        result.append(entropy)
        # histogram:
        ## plot figure on every n frequency, subject and brain region :
        if col % 3 == 0 and int(subj_id) % 50 == 0 and int(roi) % 20 == 0:
            plt.hist(spectrum[col], bins = 30)
            figure_name = file.replace('.csv', '_freq_{:03}'.format(col) + diagnosis + '.png')
            figure_name = figure_name.replace(input_dir, output_dir)
            plt.savefig(figure_name)
            plt.clf()
    
        # break
    entropy_list[int(subj_id)][int(roi)] = result
    # outfile_name = outfile_name_header + '_subj_' + subj_id + '_roi_' + roi + '.csv'
    # np.savetxt(output_dir + outfile_name, np.array(result), delimiter = ',')
    
    # break
    
print('finished!')

ocd_out03_spectrum_power_hc/spectrum_subj000_roi_000.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_001.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_002.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_003.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_004.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_005.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_006.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_007.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_008.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_009.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_010.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_011.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_012.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_013.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_014.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_015.csv
ocd_out03_spectrum_power_hc/spectrum_subj000_roi_016.csv
ocd_out03_spectrum_power_hc/spe

<Figure size 432x288 with 0 Axes>

In [48]:
## save result as subject by roi .csv files.

num_subject = len(entropy_list)
num_roi = len(entropy_list[0])
num_freq = len(entropy_list[0][0]) 

for i in range(num_freq):
    res = [[np.nan] * num_roi for _ in range(num_subject)]
    for subj in range(num_subject):
        for roi in range(num_roi):
            if entropy_list[subj][roi] and entropy_list[subj][roi][i]:
                res[subj][roi] = entropy_list[subj][roi][i]
                df = pd.DataFrame(data = res)
                df = pd.concat([subject_info, df], axis = 1)
                
    outfile_name = outfile_name_header + '_freq_{:02}'.format(i) + '.csv'
    # np.savetxt(output_dir + outfile_name, np.array(df), delimiter = ',')
    df.to_csv(output_dir + outfile_name)
    
print('finished!')  

In [47]:
entropy_list

[[[3.3628213533219893,
   3.4336193731920397,
   3.6378766239473475,
   3.4845462216644525,
   3.675913281213871,
   3.4416519698223316,
   3.374467723019922,
   3.466599649197544,
   3.4974187531411145,
   3.471446055714431],
  [3.3415834177481987,
   3.4223805330290715,
   3.4595303317379797,
   3.5665880325631196,
   3.4866040847360664,
   3.3324261866588833,
   3.3331985524940126,
   3.429959478004945,
   3.459817455691219,
   3.366319960583827],
  [3.47766096710333,
   3.4165016602586804,
   3.627116494156532,
   3.4168487426467147,
   3.3741024921905276,
   3.614491723869784,
   3.598093806699571,
   3.584922115489408,
   3.4767160207451413,
   3.483828688760231],
  [3.5303395396667385,
   3.370229879819761,
   3.4529406877240194,
   3.6236707415504084,
   3.6484254328273975,
   3.468214265644638,
   3.3090492143639625,
   3.673637860038432,
   3.2254908562325975,
   3.574626193303156],
  [3.11469518999242,
   3.5189425619090833,
   3.2542754308890176,
   3.5479153673007824,
   3